# 信号処理用ノートブック

## ライブラリのインポート

In [1]:
import json
import numpy as np
import pandas as pd
import collections
from tqdm import tqdm
from tslearn.clustering import KShape

from lib import Util

/home/csi/minelab-agri-platform-ml/.venv/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


## 設定ファイルの読み込み

In [2]:
# 設定ファイルの読み込み
with open(f"{Util.get_root_dir()}/../config/config.json", "r") as f:
    config = json.load(f)

## 信号処理パイプライン

In [3]:
class SignalProcessor:
    """信号処理クラス"""
    def __init__(self, df):
        self.df = df

    def remove_all_zero_col(self, inplace:bool=False):
        """全て0の列を削除"""
        try:
            df = self.df.copy()

            # 全て0の列を削除
            for col in df.columns:
                if (df[col] == 0).all():
                    df = df.drop(col, axis=1)

            # inplace=Trueの場合は元のdfを更新
            if inplace:
                self.df = df
                return self.df
            else:
                return df
        except Exception as e:
            raise e

    def remove_all_value_col(self, inplace:bool=False):
        """全て同じ値の列を削除"""
        try:
            df = self.df.copy()

            # 全て同じ値の列を削除
            for col in df.columns:
                if len(df[col].unique()) == 1:
                    df = df.drop(col, axis=1)

            # inplace=Trueの場合は元のdfを更新
            if inplace:
                self.df = df
                return self.df
            else:
                return df
        except Exception as e:
            raise e

    def hampel_filter(self, window_size:int=5, alpha:float=3.0, inplace:bool=False):
        """hampelフィルターを全ての列に適用"""
        try:
            df = self.df.copy()

            # hampelフィルターを全ての列に適用
            for col in df.columns:
                df[col] = self._hampel_filter_col(df[col].values, window_size, alpha)

            # inplace=Trueの場合は元のdfを更新
            if inplace:
                self.df = df
                return self.df
            else:
                return df
        except Exception as e:
            raise e

    def _hampel_filter_col(self, array:np.ndarray, window_size:int, alpha:float) -> np.ndarray:
        """
        Hampelフィルター

        params
        ------
        array: np.ndarray
            フィルターをかける配列
        window_size: int
            ウィンドウサイズ
        alpha: float
            閾値

        return
        ------
        np.ndarray
            フィルター後の配列
        """
        new_array   = array.copy()   # フィルター後の配列
        n           = len(array)     # 配列の長さ
        half_window = window_size//2 # ウィンドウサイズの半分

        # 各要素についてフィルターをかける
        for i in range(n):
            # ウィンドウの範囲を取得
            start  = max(0, i-half_window)   # 0以下にならないようにmaxを取る
            end    = min(n, i+half_window+1) # n以上にならないようにminを取る
            kernel = array[start:end]        # ウィンドウの範囲を取得

            # 中央値と尺度統計量を計算
            median = np.median(kernel)                 # 中央値
            mad    = np.median(np.abs(kernel-median))  # MAD (中央値絶対偏差)
            std    = 1.4826*mad                        # Hampelフィルターのスケールファクター

            # 外れ値の検出と置き換え
            if np.abs(array[i]-median) > alpha*std:
                new_array[i] = median
        return new_array

    def difference_filter(self, stride:int=1, inplace:bool=False):
        """差分フィルター"""
        try:
            df = self.df.copy()

            # 差分フィルター
            df = df.diff(stride)
            df = df.dropna(axis=0)
            df = df.reset_index(drop=True)

            # inplace=Trueの場合は元のdfを更新
            if inplace:
                self.df = df
                return self.df
            else:
                return df
        except Exception as e:
            raise e

    def standardize(self, inplace:bool=False):
        """標準化"""
        try:
            df = self.df.copy()

            # 標準化
            df = (df - df.mean()) / df.std()

            # inplace=Trueの場合は元のdfを更新
            if inplace:
                self.df = df
                return self.df
            else:
                return df
        except Exception as e:
            raise e

    def kshape_clustering(self, n_clusters:int=1, inplace:bool=False):
        """KShapeクラスタリング"""
        try:
            df = self.df.copy()

            # KShapeクラスタリング
            kshape = KShape(n_clusters=n_clusters, random_state=3407)
            kshape_base = kshape.fit(df.T.values)
            cnt = collections.Counter(kshape_base.labels_)
            cluster_labels_KS = {}
            for k in cnt:
                cluster_labels_KS['cluster-{}'.format(k)] = cnt[k]
            centroids = pd.DataFrame()
            for i in range(len(kshape_base.cluster_centers_)):
                centroids = pd.concat([centroids, pd.Series(kshape_base.cluster_centers_[i].reshape(len(kshape_base.cluster_centers_[i])))], axis=1)

            # inplace=Trueの場合は元のdfを更新
            if inplace:
                self.df = centroids
                return self.df
            else:
                return centroids
        except Exception as e:
            raise e

## 信号処理

In [4]:
# 共通ファイルを取得
common_file = Util.get_common_files(path_list=[f"{Util.get_root_dir()}/../data/csv-data/{field_device}/amp/" for field_device in config["FieldDevice"]["Pcap"]])

In [5]:
# 各ファイルに対して信号処理を適用
for field_device in sorted(config["FieldDevice"]["Pcap"]):
    for file_name in tqdm(common_file):
        # ファイルのパスを取得
        file_path = f"{Util.get_root_dir()}/../data/csv-data/{field_device}/amp/{file_name}"
        # データを読み込み（TimeカラムをIndexに設定）
        df = pd.read_csv(file_path, index_col=0).set_index("Time", drop=True)

        # 信号処理を適用
        sp = SignalProcessor(df)
        ## 全て0の列を削除
        sp.remove_all_zero_col(inplace=True)
        ## hampelフィルターを適用
        sp.hampel_filter(inplace=True)
        ## 全て同じ値の列を削除
        sp.remove_all_value_col(inplace=True)
        ## 標準化
        sp.standardize(inplace=True)

        # データを保存
        Util.create_path(f"{Util.get_root_dir()}/../data/preprocessed-data/{field_device}/amp/")
        sp.df.to_csv(f"{Util.get_root_dir()}/../data/preprocessed-data/{field_device}/amp/{file_name}", index=True)

100%|██████████| 4/4 [00:01<00:00,  2.20it/s]
